In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/nyc_bike_accidents.csv')
pd.options.display.max_columns = 50

In [ ]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])


df['YEAR'] = df['CRASH DATE'].dt.strftime('%Y')
df

In [ ]:
df.groupby('YEAR').sum('NUMBER OF PERSONS KILLED')

In [ ]:
# Get only the data from 2022
df_2022 = df[df['YEAR'] == '2022']

# Get only the columns we need
df_2022 = df_2022[['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF CYCLIST INJURED']]

# And create new columns that will be used for visualization
df_2022['ACCIDENT TYPE'] = 0

df_2022.loc[df_2022['NUMBER OF CYCLIST INJURED'] > 0, 'ACCIDENT TYPE'] = 'Injured'
df_2022.loc[df_2022['NUMBER OF CYCLIST KILLED'] > 0, 'ACCIDENT TYPE'] = 'Fatal'

df_2022['INJURED OR DEATH'] = df_2022['NUMBER OF CYCLIST KILLED'] + df_2022['NUMBER OF CYCLIST INJURED']

df_2022.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)


In [ ]:
df_2022.to_csv('data/nyc_bike_crashes_2022.csv')

In [ ]:
accidents = pd.read_csv('data/nyc_bike_accidents.csv')

bikers = pd.read_csv('data/nyc_bikerides_numbers.csv')

In [ ]:
accidents['CRASH DATE'] = pd.to_datetime(accidents['CRASH DATE'])

accidents['YEAR'] = accidents['CRASH DATE'].dt.strftime('%Y')

accidents_year = accidents.groupby('YEAR').sum('NUMBER OF PERSONS KILLED')
accidents_year = accidents_year[['NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED']]
accidents_year

In [ ]:
accidents_year = accidents_year.reset_index()

accidents_year.rename(columns={"YEAR": "year", "NUMBER OF CYCLIST INJURED": "injured", "NUMBER OF CYCLIST KILLED": "killed"}, inplace=True)

accidents_year

In [ ]:
bikers['year'] = bikers['Unnamed: 0']
bikers = bikers[['year', 'Total Daily Cycling Trips']]
bikers

In [ ]:
bikers.drop([13], inplace=True)
bikers

In [ ]:
bikers_accidents = pd.merge(accidents_year, bikers, on=['year'], how='left')

bikers_accidents['injury_rate'] = bikers_accidents['injured'] / bikers_accidents['Total Daily Cycling Trips']
bikers_accidents['fatality_rate'] = bikers_accidents['killed'] / bikers_accidents['Total Daily Cycling Trips']


bikers_accidents.rename({'Total Daily Cycling Trips': 'total_daily_bikerides'}, axis=1, inplace=True)

bikers_accidents.to_csv('data/accidents_development.csv')